# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Queuing model and discrete event simulation of a gas station

Recall the introduction to queuing models and discrete event simulators from the last class: [link](https://t-square.gatech.edu/access/content/group/gtc-239f-fc11-5690-9dae-2dc96b59f372/cx4230-sp17--22--queueing.pdf). In this notebook, you will implement it.

## Exponential random numbers

Recall that in a queuing model, it is common to assume that customer interarrival times and service times are independent and identically distributed random variables. Classically, the most commonly assumed distribution is _exponential_.

More specifically, an exponentially distributed random variable $X \sim \mathcal{E}(\lambda)$ has the probability density function,

$$
  f_X(x) = \lambda \cdot \exp\left(-\frac{x}{\lambda}\right),
$$

where $\lambda$ is the mean of the distribution.

Using Numpy, these are easy to generate using the function, [`numpy.random.exponential()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.exponential.html).

Here is a quick demo.

In [ ]:
from numpy.random import exponential, seed

X_MEAN = 10.0
X_COUNT = 5
x_values = exponential (X_MEAN, X_COUNT)

print ("X ~ Exp(%g):" % X_MEAN)
for (i, x_i) in enumerate (x_values):
    print ("  X_%d = %g" % (i, x_i))

As a sanity check, let's generate a large number of values and compare the sample mean to the desired (true) mean.

In [ ]:
from numpy import mean

# Demo
N_BIG = 1000
big_mean = mean (exponential (X_MEAN, N_BIG))
print ("\nSample mean of %d values: %g (true mean: %g)" % (N_BIG, big_mean, X_MEAN))

## Priority queues

To maintain the future event list, you need some kind of priority queue data structure. One classical choice is to use a heap, for which there is a standard implementation in Python: [`heapq` link](http://www.bogotobogo.com/python/python_PriorityQueue_heapq_Data_Structure.php). The `heapq` interface includes the following operations:

* `heapify(h)`: Convert an unordered input collection `h` into a min-heap *in-place*.
* `heappush(h, x)`: Insert element `x` into the heap `h`.
* `x = heappop(h)`: Remove the minimum element from the heap `h`.

Here's a quick demo.

In [ ]:
from heapq import heappush, heappop, heapify

In [ ]:
# Method 1: Convert any Python list into a heap
h1 = list (x_values)
print ("Initial values:", h1)

heapify (h1)
print ("\nHeapified:", h1)

print ("\nExtracting mins...")
for i in range (len (h1)):
    print (i, ":", heappop (h1), "[heap: {}]".format (h1))

In [ ]:
# Method 2: Insert values into the heap one at a time

print ("Inserting...")
h2 = []
for (i, x_i) in enumerate (x_values):
    print (i, ":", x_i)
    heappush (h2, x_i)
    
print ("\nHeap:", h2)
    
print ("\nExtracting minima...")
for i in range (len (h2)):
    print (i, ":", heappop (h2))

## A generic discrete event simulation engine

We can build a simple, generic discrete event simulation engine. This engine manages the future event list, which you'll recall is a priority queue of timestamped events. It continually removes the event with the lowest timestamp and processes it.

Suppose we represent an event by a tuple, `(t, e)`, where `t` is the event's timestamp and `e` is an event handler. An event handler is simply a callback function. Let's suppose that this function takes two arguments, `e(t, s)`, where `t` is (again) the timestamp and `s` is the system state, encoded in an application-specific way. (That means the application developer decides what sort of object `s` represents.) When `e(t, s)` executes, it might update the state `s`.

**Exercise 1** (2 points). Complete the following function, which implements a generic discrete event simulation engine. The future events list is a heap named `events`. The initial system state is `initial_state`; the starter code below makes a copy of this state as a variable `s`, which your simulator is allowed to modify (and, as the scaffolding code shows, will eventually return).

In [ ]:
from copy import deepcopy

def simulate (events, initial_state, trace=True):
    s = deepcopy (initial_state)
    
    print ("\nFuture event list:\n%s" % str (events))
    print ("\nt=0: %s" % str (s))
    
    if trace:
        s_all = []
        
    # `s` is the current simulation state
    while events:
        # Get event and process it, updating `s`
        # YOUR CODE HERE
        raise NotImplementedError()
        
        if trace:
            s_all.append (deepcopy (s))
            
    if trace:
        return s_all
    else:
        return s

In [ ]:
# Let the state be a dictionary with two fields:
# 't', which stores a timestamp, and `c`, which
# stores an integer count.
def test_event_add (t, s):
    s['t'] = t
    s['c'] += 1
    
def test_event_sub (t, s):
    s['t'] = t
    s['c'] -= 1

s_test_initial = {'t': 0, 'c': 0}
print ("Initial state:", s_test_initial)

events_test = [(1.23, test_event_add),
               (4.72, test_event_add),
               (5.1, test_event_add),
               (0.75, test_event_sub),
               (6.2, test_event_sub)]
heapify (events_test)
s_test_all = simulate (events_test, s_test_initial)
print ("\n==> Events processed:", s_test_all)

assert [s_test_all[j]['c'] for j in range (len (s_test_all))] == [-1, 0, 1, 2, 1]
assert [s_test_all[j]['t'] for j in range (len (s_test_all))] == [0.75, 1.23, 4.72, 5.1, 6.2]
print ("\n(Passed.)")

## Instantiating the simulator

For the gas station model, suppose we have an existing trace of five cars whose _interarrival times_ are as follows:

In [ ]:
car_interarrival_times = [3.6, 5.6, 10.5, 4.1, 1.8] # Minutes
print ("Interrival times (in minutes) of all cars:\n", car_interarrival_times)

Furthermore, let's suppose that the pumping times and shopping times are exponentially distributed with mean times of 5 minutes and 10 minutes, respectively.

In [ ]:
# Number of customers (cars)
NUM_CARS = len (car_interarrival_times)

# Event parameters
MEAN_PUMPING_TIME = 5.0 # minutes
MEAN_SHOPPING_TIME = 10.0 # minutes

Recall that the state consists of the logical simulation time (`now`) and three state variables: `AtPump`, `AtStore`, and `PumpFree`. Let's create this state.

In [ ]:
state = {'AtPump': 0          # no. cars at pump or waiting
         , 'AtStore': 0       # no. cars at store
         , 'PumpFree': True   # True <==> pump is available
        }

Let's represent an _event_ as a tuple, `(t, e)`, where `t` is the timestamp of the event and `e` is an event handler, implemented as a Python function.

If the future event list is stored in a **global** priority queue called `events`, the following function will insert an event into that queue.

In [ ]:
def schedule (t, e, debug=True):
    """
    Schedules a new event `e` at time `t`.
    """
    global events
    if debug: print ("  ==> '%s' @ t=%g" % (e.__name__, t))
    heappush (events, (t, e))

**Defining events: arrivals.** Per the slides from class, let's suppose our simulator will have three possible events. (These event handlers take as input the event timestamp, `t`, and the current simulation state, `s`.)

- `arrives(t, s)`: When the simulation is in state `s` at time `t`, a car arrives.
- `finishes(t, s)`: When the simulation is in state `s` at time `t`, the car occupying the gas pump finishes pumping gas.
- `departs(t, s)`: When the simulation is in state `s` at time `t`, a car at the store leaves the gas station.

Below, we define the event handler for car arrival events. Compare it to the sample code in the slides from class. (See, in particular, slide/page 29.) In this case, recall that we pre-generated all car interarrival times in one of the earlier code cells; therefore, the implementation below skips one of the steps that is shown in the slides.

In [ ]:
def arrives (t, s):
    """
    Processes an arrival event at time `t` for a system in state `s`.
    Schedules a pumping event if the pump is free. Returns the new
    system state.
    """
    s['AtPump'] += 1
    if s['PumpFree']:
        s['PumpFree'] = False
        schedule (t + exponential (MEAN_PUMPING_TIME), finishes) # `finishes()` TBD, below.
    return s

**Exercise 2** (2 points). Create an initial future events list by converting the raw interarrival times into arrival events and inserting them into the future events list.

In [ ]:
# Hint: This function may prove useful
from numpy import cumsum

events = []  # Future event list, initially empty

def create_arrival_events (interarrival_times):
    # YOUR CODE HERE
    raise NotImplementedError()
    
create_arrival_events (car_interarrival_times)
print ("\nContents of `events[:]`:")
for (i, event) in enumerate (events):
    print ("[%d] t=%g: %s" % (i, event[0], event[1].__name__))

In [ ]:
# Test code
assert len (events) == 5
assert all ([events[i][0] < events[i+1][0] for i in range (len (events)-1)])
assert events[-1][0] == sum (car_interarrival_times)
print ("\n(Passed.)")

**Exercise 3** (4 points). Implement `finishes()` and `departs()`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
state_0 = {'AtPump': 0          # no. cars at pump or waiting
           , 'AtStore': 0       # no. cars at store
           , 'PumpFree': True   # True <==> pump is available
          }

events = []
create_arrival_events (car_interarrival_times)

seed (1)
states_all = simulate (events, state_0)

assert [states_all[i]['AtPump'] for i in range (len (states_all))] == [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
assert [states_all[i]['AtStore'] for i in range (len (states_all))] == [0, 1, 1, 2, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 0]
assert [states_all[i]['PumpFree'] for i in range (len (states_all))] == [False, True, False, True, True, True, False, True, True, False, True, False, True, True, True]
print ("\n(Passed.)")

In [ ]:
# IGNORE this code:
for f in ['AtPump', 'AtStore', 'PumpFree']:
    values = []
    for s in states_all:
        values.append (str (s[f]))
    print ('assert [states_all[i][\'{}\'] for i in range (len (states_all))] == [{}]'.format (f, ', '.join (values)))